# Cluster images

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt
import os
import cv2  # Ensure OpenCV is installed

# Load the CSV file
df = pd.read_csv('NML_Descriptions_to_score.csv', encoding='utf-8')

# Assuming your CSV has a column 'Extracted Image Path' that contains image paths
image_paths = df['Extracted Image Path'].dropna().tolist()  # Drop any NaN values

# Function to extract features from an image using a pre-trained model
def extract_image_features(image_path):
    if not os.path.exists(image_path):  # Check if the image file exists
        return None  # Return None if image is not found
    try:
        img = cv2.imread(image_path)  # OpenCV for image loading
        img = cv2.resize(img, (224, 224))  # Resize image to match model input
        img_array = np.array(img, dtype=np.float32)  # Convert image to array
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
        features = model.predict(img_array, verbose=0)  # Suppress verbose output
        return features.flatten()  # Flatten features for clustering
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None  # Return None if there is an issue with the image

# Load the pre-trained model for feature extraction
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Extract features from all images
features = []
valid_image_paths = []
for path in image_paths:
    feature = extract_image_features(path)
    if feature is not None:  # Only append valid features
        features.append(feature)
        valid_image_paths.append(path)  # Keep track of valid image paths

# Check if features were extracted successfully
if len(features) == 0:
    print("No valid features extracted. Check your image paths.")
else:
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=50)  # Reduce to 50 components
    reduced_features = pca.fit_transform(features)

    # Elbow Method (for determining optimal k)
    distortions = []
    K_range = range(1, 11)  # Try different values of k (1 to 10)
    for k in K_range:
        kmeans = KMeans(n_clusters=k, verbose=0)  # Suppress verbose output
        kmeans.fit(reduced_features)
        distortions.append(kmeans.inertia_)

    # Plot Elbow Method
    plt.figure(figsize=(8, 6))
    plt.plot(K_range, distortions, marker='o')
    plt.title('Elbow Method for Optimal k (Best K: Elbow point)')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Distortion (Inertia)')
    plt.show()

    # Silhouette Score (for evaluating the clustering quality)
    silhouette_scores = []
    for k in K_range[1:]:  # Start from 2 clusters (since silhouette score requires at least 2 clusters)
        kmeans = KMeans(n_clusters=k, verbose=0)  # Suppress verbose output
        kmeans.fit(reduced_features)
        score = silhouette_score(reduced_features, kmeans.labels_)
        silhouette_scores.append(score)

    # Plot Silhouette Scores
    plt.figure(figsize=(8, 6))
    plt.plot(K_range[1:], silhouette_scores, marker='o', color='g')
    plt.title('Silhouette Scores for Different k. (A higher score (closer to 1) indicates better-defined clusters)')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.show()

    # Davies-Bouldin Index (for evaluating how well-separated the clusters are)
    davies_bouldin_scores = []
    for k in K_range[1:]:
        kmeans = KMeans(n_clusters=k, verbose=0)  # Suppress verbose output
        kmeans.fit(reduced_features)
        score = davies_bouldin_score(reduced_features, kmeans.labels_)
        davies_bouldin_scores.append(score)

    # Plot Davies-Bouldin Index Scores
    plt.figure(figsize=(8, 6))
    plt.plot(K_range[1:], davies_bouldin_scores, marker='o', color='r')
    plt.title('Davies-Bouldin Index for Different k. (A lower score indicates well-separated clusters)')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Davies-Bouldin Index')
    plt.show()


# Take a decition for optimal number of clusters (k) based on the Elbow method, Davies-Bouldin Index, and Silhouette Score

In [24]:
# Perform final KMeans clustering with optimal k (based on your chosen evaluation metric)
optimal_k = 4  # For example, if k=3 seems optimal from your evaluation
kmeans = KMeans(n_clusters=optimal_k, verbose=0)  # Suppress verbose output
kmeans.fit(reduced_features)

# Map images to clusters
image_cluster_map = {valid_image_paths[i]: kmeans.labels_[i] for i in range(len(valid_image_paths))}

# Optionally, save the mapping to a new CSV file
df['Cluster'] = df['Extracted Image Path'].map(image_cluster_map)
df.to_csv('clustered_images_with_metrics.csv', index=False)
print("Clustering results saved to 'clustered_images_with_metrics.csv'.")

Clustering results saved to 'clustered_images_with_metrics.csv'.


# Display a subset of images in each cluster

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageOps
from IPython.display import display
import IPython.display as ipd

# Load the CSV file
csv_path = "clustered_images_with_metrics.csv"
df = pd.read_csv(csv_path)

# Ensure correct column names
image_column = "Extracted Image Path"
cluster_column = "Cluster"

# Filter out rows where the cluster label is NaN
df = df[df[cluster_column].notna()]

# Get unique non-empty clusters
non_empty_clusters = [cluster for cluster in df[cluster_column].unique() if df[df[cluster_column] == cluster][image_column].notna().sum() > 0]

# Define a dictionary of background colors for each cluster
cluster_colors = {
    0: (220, 200, 200),  # Light mint green for Cluster 1
    1: (255, 240, 200),  # Light yellow for Cluster 2
    2: (200, 220, 240),  # Light blue for Cluster 3
    3: (220, 240, 240),  # Soft turquoise for Cluster 4
    4: (240, 240, 200),  # Light cream for Cluster 5
    5: (240, 240, 240),  # Light lavender for Cluster 6
    6: (200, 240, 220),  # Light aqua for Cluster 7
    7: (240, 230, 200),  # Light beige for Cluster 8
    8: (220, 220, 240),  # Light periwinkle for Cluster 9
    # Add more clusters and colors as needed
}

# Function to pad images while keeping aspect ratio
def pad_image(img, size=256, bg_color=(220, 200, 200)):  # Default background color
    """Resizes an image to fit within a square while maintaining aspect ratio, padding with a background color."""
    img.thumbnail((size, size), Image.Resampling.LANCZOS)  # Resize while maintaining aspect ratio
    new_img = Image.new("RGB", (size, size), bg_color)  # Create a square canvas
    x_offset = (size - img.size[0]) // 2  # Center the image
    y_offset = (size - img.size[1]) // 2
    new_img.paste(img, (x_offset, y_offset))
    return new_img

# Function to save and display images for a given cluster
def save_and_display_images_for_cluster(cluster_df, cluster_label, output_dir="output_images"):
    images = cluster_df[image_column].dropna().tolist()[:9]  # Get first 9 images

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Set the background color for the cluster
    bg_color = cluster_colors.get(int(cluster_label), (220, 200, 200))  # Default to mint green if not defined

    fig, axes = plt.subplots(3, 3, figsize=(9, 9))
    fig.suptitle(f"Cluster {int(cluster_label)}", fontsize=16)  # Force integer display

    for ax, img_path in zip(axes.flatten(), images):
        if os.path.exists(img_path):
            img = Image.open(img_path)
            img = pad_image(img, bg_color=bg_color)  # Use the cluster's background color
            ax.imshow(img)
            ax.set_title(os.path.basename(img_path), fontsize=8)
        else:
            ax.set_title("Not Found", fontsize=8)
        ax.axis("off")

    # Hide any empty subplots if <9 images
    for i in range(len(images), 9):
        axes.flatten()[i].axis("off")

    # Save the figure as a JPG file
    cluster_filename = os.path.join(output_dir, f"cluster_{int(cluster_label)}.jpg")
    plt.tight_layout()
    plt.savefig(cluster_filename, format="jpg", dpi=300)
    plt.show()  # Display the figure in the notebook

# Loop through the original clusters (after removing NaN entries)
original_clusters = sorted(df[cluster_column].unique())

for original_cluster in original_clusters:  # Use original cluster labels
    cluster_df = df[df[cluster_column] == original_cluster]
    save_and_display_images_for_cluster(cluster_df, original_cluster)
